In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
from transformers import TFAutoModel,AutoTokenizer

In [3]:
model=TFAutoModel.from_pretrained("bert-base-uncased")
# Remember bert based has 12 Encoded Layer 
# While the bert large has 24 Encoded Layer

#uncased used here means it is not case sensetive

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [5]:
tokenizer=AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
input=tokenizer("Hello World")
input

{'input_ids': [101, 7592, 2088, 102], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}

In [13]:
input=tokenizer(["Hello World","Hi how are you"],padding=True,truncation=True,
               return_tensors="tf")
input
# Truncation will limit the word size to be 500 words for the nlp task
# return_tensors helps to convert the list into the tensors

{'input_ids': <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[ 101, 7592, 2088,  102,    0,    0],
       [ 101, 7632, 2129, 2024, 2017,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[1, 1, 1, 1, 0, 0],
       [1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [14]:
output=model(input)
output

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(2, 6, 768), dtype=float32, numpy=
array([[[-0.16888323,  0.13606343, -0.13940047, ..., -0.6251125 ,
          0.05217249,  0.36714536],
        [-0.36327457,  0.14121887,  0.8799876 , ...,  0.10433066,
          0.28875753,  0.3726792 ],
        [-0.6985945 , -0.6987972 ,  0.06450225, ..., -0.22103679,
          0.00986902, -0.5939791 ],
        [ 0.8309831 ,  0.12366767, -0.15119001, ...,  0.10309552,
         -0.6779269 , -0.2628521 ],
        [-0.4026663 , -0.01928258,  0.57325006, ..., -0.2065685 ,
          0.02338566,  0.20126341],
        [-0.6228407 , -0.27453506,  0.18117577, ..., -0.12944894,
         -0.03839066, -0.05733148]],

       [[ 0.09286553, -0.02636376, -0.12239317, ..., -0.2106353 ,
          0.17386353,  0.17250979],
        [ 0.4074204 , -0.05930945,  0.5523465 , ..., -0.67905635,
          0.6555742 , -0.29456428],
        [-0.211553  , -0.6858638 , -0.4628075 , ...,  0.15278497

In [15]:
from datasets import load_dataset

In [16]:
emotions=load_dataset("Setfit/emotion")

README.md:   0%|          | 0.00/194 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl:   0%|          | 0.00/2.23M [00:00<?, ?B/s]

validation.jsonl:   0%|          | 0.00/276k [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/279k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [17]:
emotions

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 2000
    })
})

In [18]:
def tokenize(batch):
    return tokenizer(batch["text"],padding=True,truncation=True)

In [19]:
emotions_encodded=emotions.map(tokenize,batched=True,batch_size=None)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [20]:
emotions_encodded

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [22]:
import tensorflow as tf

# Function to convert dataset into TensorFlow tensors
def convert_to_tf(dataset):
    input_ids = tf.convert_to_tensor(dataset["input_ids"])
    attention_mask = tf.convert_to_tensor(dataset["attention_mask"])
    labels = tf.convert_to_tensor(dataset["label"])
    
    return ({"input_ids": input_ids, "attention_mask": attention_mask}, labels)

# Convert train, validation, and test sets
train_dataset = tf.data.Dataset.from_tensor_slices(convert_to_tf(emotions_encodded["train"]))
val_dataset = tf.data.Dataset.from_tensor_slices(convert_to_tf(emotions_encodded["validation"]))
test_dataset = tf.data.Dataset.from_tensor_slices(convert_to_tf(emotions_encodded["test"]))

# Batch the dataset for training
BATCH_SIZE = 32
train_dataset = train_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)


In [23]:
inp,out=next(iter(train_dataset)) # a batch from the train dataset
print(inp,"\n\n",out)

{'input_ids': <tf.Tensor: shape=(32, 87), dtype=int32, numpy=
array([[  101,  1045,  2134, ...,     0,     0,     0],
       [  101,  1045,  2064, ...,     0,     0,     0],
       [  101, 10047,  9775, ...,     0,     0,     0],
       ...,
       [  101,  1045,  2514, ...,     0,     0,     0],
       [  101,  1045,  2131, ...,     0,     0,     0],
       [  101,  1045,  3342, ...,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(32, 87), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>} 

 tf.Tensor([0 0 3 2 3 0 5 4 1 2 0 1 3 0 1 1 0 0 0 4 3 4 1 1 3 0 0 0 3 1 1 4], shape=(32,), dtype=int32)


In [36]:
class BERTForClassification(tf.keras.Model):
    def __init__(self,bert_model,num_classes):
        super().__init__()
        self.bert=bert_model
        self.fc=tf.keras.layers.Dense(num_classes,activation="softmax")

    def call(self,inputs):
        x=self.bert(inputs)[1]
        return self.fc(x)

In [37]:
classifier= BERTForClassification(model,num_classes=6)
classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [38]:
history=classifier.fit(
    train_dataset,
    epochs=3
)

Epoch 1/3
500/500 [==============================] - 205s 345ms/step - loss: 0.6750 - accuracy: 0.7573
Epoch 2/3
500/500 [==============================] - 172s 345ms/step - loss: 0.1706 - accuracy: 0.9320
Epoch 3/3
500/500 [==============================] - 172s 345ms/step - loss: 0.1164 - accuracy: 0.9479


In [39]:
classifier.evaluate(test_dataset)

63/63 [==============================] - 9s 97ms/step - loss: 0.1718 - accuracy: 0.9245


[0.1718362271785736, 0.9244999885559082]